## Using complete as Output Mode

Let us see few examples using `complete` as output mode. It is typically used on top of data frames which involves aggregations as part of transformations.
* If we try to use `append` with aggregations as part of transformations, it will fail with error.

Launch Pyspark using below commands and run Spark Structured Streaming Code.

**Using Pyspark2**

```
export PYSPARK_PYTHON=python3

pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark3**

```
export PYSPARK_PYTHON=python3

pyspark3 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

* Let us read the data, apply aggregations and write to console using `complete` as output mode.

```python
spark.conf.set('spark.sql.shuffle.partitions', '2')

import socket
hostname = socket.gethostname()

log_messages = spark. \
    readStream. \
    format("socket"). \
    option("host", hostname). \
    option("port", 9000). \
    load()

from pyspark.sql.functions import split, count, lit

department_count = log_messages. \
    filter(split(split('value', ' ')[6], '/')[1] == 'department'). \
    select(split(split('value', ' ')[6], '/')[2].alias('department')). \
    groupBy('department'). \
    agg(count(lit(1)).alias('department_count'))

department_count. \
    writeStream. \
    outputMode("complete"). \
    format("console"). \
    option('truncate', 'false'). \
    trigger(processingTime='5 seconds'). \
    start()
```